<a href="https://colab.research.google.com/github/gtxdatathon/GTX-Geothermal-Datathon-Marcelo/blob/main/Data_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GTX Bootcamp - Data Engineering 
<small>by [Marcelo Guarido](https://www.linkedin.com/in/mguarido/)</small>
<br>

In this notebook, the focus will be in the data preparation prior to modeling.

## Summary

* Data preparation
    * Cleaning
    * Imputation (with mean/median)
    * Feature engineering

## Load useful packages

For the well log data analyses, the main packages used are:
* Pandas
* Matplotlib
* Seaborn

Also, from the file "utils.py", load the log visualization function.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.metrics import confusion_matrix
import itertools

mpl.rcParams['figure.figsize'] = (20.0, 10.0)
inline_rc = dict(mpl.rcParams)

In [ ]:
# If seaborn package is missing, uncomment and run the following line:
# ! pip install seaborn

## Taking a look at the data

This data is from the [2016 ML contest](https://github.com/seg/2016-ml-contest), with the focus for *Facies Classification*.

The provided data is a CSV file with well logs information from different wells. There are 5 well logs and 2 indicators:

* Gamma ray (GR)
* Resistivity (ILD\_log10)
* Photoelectric effect (PE)
* Neutron-density porosity difference (DeltaPHI)
* Average neutron-density porosity (PHIND)
* Nonmarine/marine indicator (NM\_M)
* Relative position (RELPOS)

The goal is to train a model able to classify 9 different types of rocks, as listed in the following table:

| Facies | Description | Label | Adjacent Facies|
| :---:  |    :---:    | :---: |      :---:     |
|   1    | Nonmarine Sandstone | SS | 2 |
|   2    | Nonmarine coarse siltstone | CSiS | 1,3 |
|   3    | Nonmarine fine siltstone | FSiS | 2 |
|   4    | Marine siltstone and shale | SiSh | 5 |
|   5    | Mudstone | MS | 4,6 |
|   6    | Wackestone | WS | 5,7,8 |
|   7    | Dolomite | D | 6,8 |
|   8    | Packstone-grainstone | PS | 6,7,9 |
|   9    | Phylloid-algal bafflestone | BS | 7,8 |

So, let's take a look at the data!

Loading the CSV file to *pandas* dataframe and cheking some of the features:

In [ ]:
data = pd.read_csv('data/facies_vectors.csv')
data.head(5)

In [ ]:
print(sorted(data['Well Name'].unique()))

In [ ]:
print(sorted(data['Facies'].unique()))

In [ ]:
print(sorted(data['Formation'].unique()))

## Features treatment

Before plotting the data, let's create the list of features names, colors, and check for missing data.

In [ ]:
features = list(data)[4:]
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']
target = 'Facies'
print(features)

Let's check for missing data in all the data frame columns:

In [ ]:
print(data.isna().sum())

Wells that contain missing data:

In [ ]:
wells_missing = list(data[data["PE"].isna()]["Well Name"].unique())
print(wells_missing)

In [ ]:
# Features per well
mpl.rcParams['figure.figsize'] = (30.0, 15.0)
inline_rc = dict(mpl.rcParams)

for w_idx, w in enumerate(np.unique(data["Well Name"])):
    ax = plt.subplot(3, 4, w_idx+1)
    hist = np.logical_not(np.any(np.isnan(data[data["Well Name"] == w][features].values), axis=0))
    plt.bar(np.arange(len(hist)), hist, color=facies_colors, align='center')
    ax.set_xticks(np.arange(len(hist)))
    ax.set_xticklabels(features)
    ax.set_yticks([0, 1])
    ax.set_yticklabels(['miss', 'hit'])
    ax.set_title(w)

## Missing values treatment - Mean fill

Initially, the missing values will be filled simply by the median of the feature.

In [ ]:
data_fill_mean = data.fillna(data.mean())
print(data_fill_mean.isna().sum())

## Plotting the logs

Let's use our function *make_facies_log_plot* to visualize the data.

In [ ]:
def make_facies_log_plot(logs, facies_colors):
    import warnings
    warnings.filterwarnings("ignore")
    #make sure logs are sorted by depth
    logs = logs.sort_values(by='Depth')
    cmap_facies = colors.ListedColormap(
            facies_colors[0:len(facies_colors)], 'indexed')
    
    ztop=logs.Depth.min(); zbot=logs.Depth.max()
    
    cluster=np.repeat(np.expand_dims(logs['Facies'].values,1), 100, 1)
    
    f, ax = plt.subplots(nrows=1, ncols=6, figsize=(8, 12))
    ax[0].plot(logs.GR, logs.Depth, '-g')
    ax[1].plot(logs.ILD_log10, logs.Depth, '-')
    ax[2].plot(logs.DeltaPHI, logs.Depth, '-', color='0.5')
    ax[3].plot(logs.PHIND, logs.Depth, '-', color='r')
    ax[4].plot(logs.PE, logs.Depth, '-', color='black')
    im=ax[5].imshow(cluster, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=1,vmax=9)
    
    divider = make_axes_locatable(ax[5])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar=plt.colorbar(im, cax=cax)
    cbar.set_label((17*' ').join([' SS ', 'CSiS', 'FSiS', 
                                'SiSh', ' MS ', ' WS ', ' D  ', 
                                ' PS ', ' BS ']))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')
    
    for i in range(len(ax)-1):
        ax[i].set_ylim(ztop,zbot)
        ax[i].invert_yaxis()
        ax[i].grid()
        ax[i].locator_params(axis='x', nbins=3)
    
    ax[0].set_xlabel("GR")
    ax[0].set_xlim(logs.GR.min(),logs.GR.max())
    ax[1].set_xlabel("ILD_log10")
    ax[1].set_xlim(logs.ILD_log10.min(),logs.ILD_log10.max())
    ax[2].set_xlabel("DeltaPHI")
    ax[2].set_xlim(logs.DeltaPHI.min(),logs.DeltaPHI.max())
    ax[3].set_xlabel("PHIND")
    ax[3].set_xlim(logs.PHIND.min(),logs.PHIND.max())
    ax[4].set_xlabel("PE")
    ax[4].set_xlim(logs.PE.min(),logs.PE.max())
    ax[5].set_xlabel('Facies')
    
    ax[1].set_yticklabels([]); ax[2].set_yticklabels([]); ax[3].set_yticklabels([])
    ax[4].set_yticklabels([]); ax[5].set_yticklabels([])
    ax[5].set_xticklabels([])
    f.suptitle('Well: %s'%logs.iloc[0]['Well Name'], fontsize=14,y=0.94)

In [ ]:
mpl.rcParams['figure.figsize'] = (8.0, 8.0)
inline_rc = dict(mpl.rcParams)

for wellName in wells_missing:
    plt.figure(figsize=(3,4))
    make_facies_log_plot(data_fill_mean[data_fill_mean['Well Name'] == wellName], facies_colors = facies_colors)

## Missing values treatment - Median fill

Initially, the missing values will be filled simply by the median of the feature.

In [ ]:
data_fill_median = data.fillna(data.median())
print(data_fill_median.isna().sum())

## Plotting the logs

Let's use our function *make_facies_log_plot* to visualize the data.

In [ ]:
for wellName in wells_missing:
    plt.figure(figsize=(3,4))
    make_facies_log_plot(data_fill_median[data_fill_median['Well Name'] == wellName], facies_colors = facies_colors)

In [ ]:
# data = data_fill_mean
data = data_fill_median

## Plotting all wells

In [ ]:
for wellName in data["Well Name"].unique():
    plt.figure(figsize=(3,4))
    make_facies_log_plot(data[data['Well Name'] == wellName], facies_colors = facies_colors)

## Removing "broken" well (data cleaning)

The well *Recruit F9* is, apparently, not so... well!!

Remove it from the data:

In [ ]:
make_facies_log_plot(data[data['Well Name'] == 'Recruit F9'], facies_colors = facies_colors)

In [ ]:
data = data[data['Well Name'] != 'Recruit F9']
print(data['Well Name'].unique())

## Missing values treatment - is mean/median fill good?

Missing data for *PE* are full in 2 wells:

+ Alexander D
+ Kimzey A

Is a median fill a good strategy to complete the missing data?

Let's check the wells.

In [ ]:
make_facies_log_plot(data[data['Well Name'] == 'ALEXANDER D'], facies_colors = facies_colors)

In [ ]:
make_facies_log_plot(data[data['Well Name'] == 'KIMZEY A'], facies_colors = facies_colors)

# Regression as Data Imputation

The *photoelectric effect* (PE) completely missing for two wells. Filling it with the median of all PE values is not an optimized solution, and can lead to a biased model.

A more suitable solution would be to train a regression model using the other logs as input and the PE as target.

<img src="figures/imputation.png" width="1000" align="center">

### **We are going to see how to use machine learning regression models for data imputation on the Bootcamp - Machine Learning on May 6th**

However, as *homework*, you can try to do it by yourself.

# Data Analysis

In [ ]:
well_names = sorted(data['Well Name'].unique())
for i in well_names:
    print(i, sorted(data[data['Well Name'] == i].Facies.unique()))

As during the regression block, define the features, target, facies colors, and facies names.

In [ ]:
features = list(data)[4:]
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_names_dict = {1 : 'SS', 2 : 'CSiS', 3 : 'FSiS', 4 : 'SiSh', 5 : 'MS', 6 : 'WS', 7 : 'D', 8 : 'PS', 9 : 'BS'}
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']
target = 'Facies'
print(features)

In [ ]:
ax = sns.pairplot(data[features + [target]], 
                  hue = target, 
                  palette = facies_colors)

## Feature augmentation

From the pairs plot, some of the logs combination showed a circular pattern over the facies classification. Machine learning classifier models like to use linear classification boundaries between the classes. An approximate circular boundary would require a very complex model, and a large dataset to be trained.

One way to avoid it is by *feature engineering*, where the original features are used to create new features that may help the predictions. Here, from the circular pattern observation, convert the features to an equivalent in polar coordinates sounds as a smart solution:

<img src="figures/polcor.png" width="800">

Having two features $X_1$ and $X_2$, the radius $r$ and angle $\phi$ are computed by the following equations:

$$r = \sqrt{{X_1}^2 + {X_2}^2}$$
$$\phi = \arctan\left(\frac{X_2}{X_1}\right)$$

In [ ]:
# Transforming the pair plots into polar coordinates
def card2polwells(data_in, features_wells):
    data_polar = data_in
    fea_red = features_wells
    name_temp = features_wells
    for fea1 in features_wells:
        del fea_red[0]
        for fea2 in fea_red:
            x = (data_in[fea1] - np.mean(data_in[fea1]))/max(data_in[fea1])
            y = (data_in[fea2] - np.mean(data_in[fea2]))/max(data_in[fea2])
            data_polar[fea1 + '_' + fea2 + '_rho'] = np.sqrt(x**2 + y**2)
            data_polar[fea1 + '_' + fea2 + '_phi'] = np.arctan2(y, x)
    
    return(data_polar)

In [ ]:
features_in = ['GR','ILD_log10','DeltaPHI','PHIND','PE']
data_polar = card2polwells(data, features_in)
data_polar.head()

Other augmentations can be done, such as the gradient of the features, the square of the features, etc.

## Gradient of the features

In [ ]:
def comp_grad(data, features, wnames, depth = 'Depth', wncol = 'Well Name'):
    for feature in features:
        data[feature + '_grad'] = 0
    
    for wname in wnames:
        dataT = data[data[wncol] == wname]
        
        for feature in features:
            dataF = dataT[feature].values
            dz = abs(dataT[depth].values[1] - dataT[depth].values[0])
            dataGrad = np.diff(dataF)/dz
            dataGrad = np.append([0], dataGrad)
            data.loc[data[data[wncol] == wname].index, feature + '_grad'] = dataGrad
            
    return data

In [ ]:
data_aug = comp_grad(data = data_polar, features = features, wnames = data["Well Name"].unique())
data_aug.head()

There are more data augmentation you can create, such as statiscal windows and clusters, but that is homework.

# $$\text{Thank you!}$$